In [2]:
import time

def count():
    print("One")
    time.sleep(1)
    print("Two")

def main():
    for _ in range(3):
        count()

if __name__ == "__main__":
    s = time.perf_counter()
    main()
    elapsed = time.perf_counter() - s
    print(f"executed in {elapsed:0.2f} seconds.")

One
Two
One
Two
One
Two
executed in 3.04 seconds.


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [16]:
import asyncio
import time

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count())

if __name__ == "__main__":
    import time
    s = time.perf_counter()
    asyncio.run(main())
    elapsed = time.perf_counter() - s
    print(f"executed in {elapsed:0.2f} seconds.")

One
One
Two
Two
executed in 1.00 seconds.


Порядок этого вывода является сердцем асинхронного ввода-вывода. Обращение к каждому из вызовов count()является одним циклом событий или координатором. Когда каждая задача достигает await asyncio.sleep(1), функция обращается к циклу событий и возвращает ему управление, говоря: «Я собираюсь заснуть на 1 секунду. Иди вперед, а пока займись чем-нибудь важным».

Хотя использование time.sleep() и asyncio.sleep() может показаться банальным, они используются в качестве дублеров для любых трудоемких процессов, связанных с временем ожидания. (Самая обыденная вещь, которую вы можете ожидать, — это sleep()вызов, который в основном ничего не делает.) То есть time.sleep() может представлять любой требующий много времени вызов блокирующей функции, тогда asyncio.sleep() как используется для замены неблокирующего вызова (но такого, который также занимает некоторое время). время завершения).

Как вы увидите в следующем разделе, преимущество ожидания чего-либо, в том числе asyncio.sleep(), заключается в том, что окружающая функция может временно передать управление другой функции, которая с большей готовностью может что-то сделать немедленно. Напротив, time.sleep() любой другой блокирующий вызов несовместим с асинхронным кодом Python, потому что он остановит все на своем пути на время ожидания.

## Правила асинхронного ввода-вывода
На данный момент уместно более формальное определение async, await и функций сопрограммы, которые они создают. Этот раздел немного запутан, но овладение async/ await играет важную роль, поэтому вернитесь к нему, если вам нужно:

Синтаксис async defвводит либо собственную сопрограмму , либо асинхронный генератор . Выражения async withи async forтакже допустимы, и вы увидите их позже.

Ключевое слово awaitпередает управление функцией обратно в цикл обработки событий. (Он приостанавливает выполнение окружающей сопрограммы.) Если Python встречает await f() выражение в области видимости g(), вот как он awaitсообщает циклу событий: «Приостановить выполнение до тех пор, пока не будет возвращено то, g () чего я жду — результат —. f() А пока пусть работает что-то еще».

In [8]:
async def f(x):
    y = await z(x)  # OK - `await` and `return` allowed in coroutines
    return y

async def g(x):
    yield x  # OK - this is an async generator

# async def m(x):
#     yield from gen(x)  # No - SyntaxError

# def m(x):
#     y = await z(x)  # Still no - SyntaxError (no `async def` here)
#     return y

In [9]:
async def nested(x):
    return x*x

In [12]:
async def f(x):
    y = await nested(x)  # OK - `await` and `return` allowed in coroutines
    return y

In [13]:
f(5)

<coroutine object f at 0x00000161CAA95740>

In [14]:
await asyncio.gather(f(1), f(2), f(3))

[1, 4, 9]

Вот один из примеров того, как асинхронный ввод-вывод сокращает время ожидания: при наличии сопрограммы makerandom(), которая продолжает создавать случайные целые числа в диапазоне [0, 10], пока одно из них не превысит пороговое значение, вы хотите, чтобы несколько вызовов этой сопрограммы не требовалось ждать, пока друг друга, чтобы завершить в последовательности. Вы можете в значительной степени следовать шаблонам из двух приведенных выше сценариев с небольшими изменениями:

In [15]:
import asyncio
import random

# ANSI colors
c = (
    "\033[0m",   # End of color
    "\033[36m",  # Cyan
    "\033[91m",  # Red
    "\033[35m",  # Magenta
)

async def makerandom(idx: int, threshold: int = 6) -> int:
    print(c[idx + 1] + f"Initiated makerandom({idx}).")
    i = random.randint(0, 10)
    while i <= threshold:
        print(c[idx + 1] + f"makerandom({idx}) == {i} too low; retrying.")
        await asyncio.sleep(idx + 1)
        i = random.randint(0, 10)
    print(c[idx + 1] + f"---> Finished: makerandom({idx}) == {i}" + c[0])
    return i

async def main():
    res = await asyncio.gather(*(makerandom(i, 10 - i - 1) for i in range(3)))
    return res

if __name__ == "__main__":
    random.seed(444)
    r1, r2, r3 = asyncio.run(main())
    print()
    print(f"r1: {r1}, r2: {r2}, r3: {r3}")

Initiated makerandom(0).
makerandom(0) == 4 too low; retrying.
Initiated makerandom(1).
makerandom(1) == 4 too low; retrying.
Initiated makerandom(2).
makerandom(2) == 0 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(1) == 7 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(2) == 4 too low; retrying.
makerandom(0) == 8 too low; retrying.
---> Finished: makerandom(1) == 10
makerandom(0) == 7 too low; retrying.
makerandom(0) == 8 too low; retrying.
makerandom(2) == 4 too low; retrying.
makerandom(0) == 7 too low; retrying.
makerandom(0) == 1 too low; retrying.
makerandom(0) == 6 too low; retrying.
---> Finished: makerandom(2) == 9
makerandom(0) == 3 too low; retrying.
makerandom(0) == 9 too low; retrying.
makerandom(0) == 7 too low; retrying.
---> Finished: makerandom(0) == 10

r1: 10, r2: 10, r3: 9


В этом миниатюрном примере пул range(3). В более полном примере, представленном ниже, это набор URL-адресов, которые необходимо запрашивать, анализировать и обрабатывать одновременно, и main()инкапсулирует всю эту процедуру для каждого URL-адреса.

In [16]:
import asyncio
import random
import time

async def part1(n: int) -> str:
    i = random.randint(0, 10)
    print(f"part1({n}) sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = f"result{n}-1"
    print(f"Returning part1({n}) == {result}.")
    return result

async def part2(n: int, arg: str) -> str:
    i = random.randint(0, 10)
    print(f"part2{n, arg} sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = f"result{n}-2 derived from {arg}"
    print(f"Returning part2{n, arg} == {result}.")
    return result

async def chain(n: int) -> None:
    start = time.perf_counter()
    p1 = await part1(n)
    p2 = await part2(n, p1)
    end = time.perf_counter() - start
    print(f"-->Chained result{n} => {p2} (took {end:0.2f} seconds).")

async def main(*args):
    await asyncio.gather(*(chain(n) for n in args))
    

if __name__ == "__main__":
    import sys
    random.seed(444)
    args = [1, 2, 3] 
    start = time.perf_counter()
    asyncio.run(main(*args))
    end = time.perf_counter() - start
    print(f"Program finished in {end:0.2f} seconds.")


part1(1) sleeping for 4 seconds.
part1(2) sleeping for 4 seconds.
part1(3) sleeping for 0 seconds.
Returning part1(3) == result3-1.
part2(3, 'result3-1') sleeping for 4 seconds.
Returning part1(1) == result1-1.
part2(1, 'result1-1') sleeping for 7 seconds.
Returning part1(2) == result2-1.
part2(2, 'result2-1') sleeping for 4 seconds.
Returning part2(3, 'result3-1') == result3-2 derived from result3-1.
-->Chained result3 => result3-2 derived from result3-1 (took 4.01 seconds).
Returning part2(2, 'result2-1') == result2-2 derived from result2-1.
-->Chained result2 => result2-2 derived from result2-1 (took 8.02 seconds).
Returning part2(1, 'result1-1') == result1-2 derived from result1-1.
-->Chained result1 => result1-2 derived from result1-1 (took 11.02 seconds).
Program finished in 11.02 seconds.
